 # CS551 Final Exam: Generative Adversarial Neural Networks



 **Authors:** 
### Final Exam

Fabio Cozzuto	Student id: 002214965

Johan Mogollon	Student id: 002359844

 **Contributions:**

 - **Fabio Cozzuto:** All code, experiments, and analysis

 - **Johan Mogollon:** All code, experiments, and analysis



 This notebook demonstrates:

 1. Padding calculation for DCGAN discriminator

 2. Data‑augmentation pipelines (`basic` vs. `deluxe`)

 3. Visualizing DCGAN samples

 4. Plotting DCGAN training losses

 5. Comparing CycleGAN outputs

# PART 1: Deep Convolutional GAN

 ## Environment Setup & Imports

In [ ]:
# Ensure Jupyter can import our GAN modules
import os, sys
sys.path.insert(0, os.path.abspath('.'))
sys.path.append('.') 

# --- Standard Libraries ---
import warnings
warnings.filterwarnings("ignore")
 

# --- Data Handling ---
import numpy as np
from PIL import Image
 

# --- PyTorch ---
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter

# --- Local Modules ---
from data_loader import get_data_loader, CustomDataSet
from models import DCGenerator, DCDiscriminator, CycleGenerator, conv, deconv, ResnetBlock
from utils import to_var, to_data, create_dir


# --- Visualization ---
import matplotlib.pyplot as plt
import imageio # For saving images


# --- Argument Parsing ---
import argparse
 

# --- Other ---
import glob
 
# Set random seed
SEED = 11
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
  torch.cuda.manual_seed(SEED)
 

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Implement Data Augmentation [10 points]

We did the implementation of the augmentations in the code:
![alt text](augmentations.png)

## Implement the Discriminator of the DCGAN [10 points]

 ### 1. Padding Calculation for DCGAN Discriminator

 **Question:** With kernel size \(K=4\) and stride \(S=2\), what padding \(P\) halves the spatial dimensions?



 **Answer:** We want each layer to reduce the spatial dimensions by a factor of 2, without clipping important features. That means we want to control the padding. So, we have the convolution output formula:

```math
O = \left \lfloor \frac{I + 2P - K}{S} \right \rfloor + 1
```
Where:
- \( I \) = input size
- \( O \) = output size
- \( K = 4 \) (kernel size)
- \( S = 2 \) (stride)
- \( P \) = padding

We want to obtain this:
```math
\text output\_size = \frac{input\_size}{2}
```
So we solve with given: 

```math
\left\lfloor \frac{I + 2P - 4}{2} \right\rfloor + 1 = \frac{I}{2}
\Rightarrow 2P = 2 \Rightarrow P = 1
```


In [5]:
# We can do the same calculation with the following code:

input_size = 64  # Example input size, this will vary per layer
kernel_size = 4
stride = 2
padding = 1
output_size = (input_size - kernel_size + 2 * padding) / stride + 1


print(f"Given kernel_size={kernel_size}, stride={stride}, the required padding is: {padding}")
print(f"Example: Input size = {input_size}, Output size = {output_size}")
 

Given kernel_size=4, stride=2, the required padding is: 1
Example: Input size = 64, Output size = 32.0


### 2. DCDiscriminator class in the models.py file

![alt text](DCDiscriminator.png)

## Implement the Generator of the DCGAN [10 points]

### 1. DCGenerator class in the models.py file

![alt text](DCGenerator.png)

 ## Experiments


### 1. Implement the DCGAN Training Loop [10 points]

Discriminator

![alt text](DCGANDisc.png)

Generator

![alt text](DCGANGen.png)

### 2. Train the DCGAN [10 points]

The next code traiin the DCGAN

In [10]:
import subprocess
def train_dcgan(data_aug_mode, num_epochs=100):
    """
    Trains the DCGAN using the vanilla_gan.py script.
    Args:
    data_aug_mode (str): The data augmentation mode ('basic' or 'deluxe').
    num_epochs (int, optional): The number of epochs to train for. Defaults to 100.
    """
    # Construct the command
    command = [
    "python",
    "vanilla_gan.py",
    f"--data_aug={data_aug_mode}",
    f"--num_epochs={num_epochs}"
    ]

    # Execute the command
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()


    # Print the output (optional, but helpful for debugging)
    print(stdout.decode())
    if stderr:
        print(stderr.decode())

In [14]:
# Basic usage
train_dcgan('basic')

Namespace(image_size=64, conv_dim=32, noise_size=100, num_epochs=100, batch_size=16, num_workers=0, lr=0.0003, beta1=0.5, beta2=0.999, data='cat/grumpifyBprocessed', data_aug='basic', ext='*.png', checkpoint_dir='./checkpoints_vanilla', sample_dir='output/./vanilla\\grumpifyBprocessed_basic', log_step=10, sample_every=200, checkpoint_every=400)
data/cat/grumpifyBprocessed\*.png
204
                    G                  
---------------------------------------
DCGenerator(
  (deconv1): Sequential(
    (0): ConvTranspose2d(100, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (deconv2): Sequential(
    (0): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (deconv3): Sequential(
    (0): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1

In [ ]:
# Deluxe augmentation
train_dcgan('deluxe') 

 ## 3. Data‑Augmentation Pipelines



 We define both **basic** and **deluxe** transforms.

 Deluxe uses a 10% up‑scale + random crop + flip :contentReference[oaicite:3]{index=3}.

In [ ]:
from data_loader import get_data_loader  # re-import for clarity

# Suppose opts.image_size is 64 for this exam
image_size = 64

basic_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5,)*3, (0.5,)*3),
])

deluxe_transform = transforms.Compose([
    transforms.Resize(int(1.1 * image_size)),
    transforms.RandomCrop(image_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,)*3, (0.5,)*3),
])

# Display example
sample_img = Image.open('data/sample.png').convert('RGB')
fig, axes = plt.subplots(1, 2, figsize=(8,4))
axes[0].imshow(np.transpose(basic_transform(sample_img).numpy(), (1,2,0)))
axes[0].set_title("Basic")
axes[0].axis('off')
axes[1].imshow(np.transpose(deluxe_transform(sample_img).numpy(), (1,2,0)))
axes[1].set_title("Deluxe")
axes[1].axis('off')
plt.show()

 ## 4. Visualizing DCGAN Samples



 We use our `DCGenerator` and display a 4×4 grid of generated images :contentReference[oaicite:4]{index=4}.

In [ ]:
# Instantiate generator
G = DCGenerator(noise_size=100, conv_dim=64).cuda()
fixed_noise = to_var(torch.randn(16, 100, 1, 1))

# Generate samples
with torch.no_grad():
    fake_images = G(fixed_noise)

# Make grid and plot
grid = make_grid(fake_images, nrow=4, normalize=True, value_range=(-1,1))
plt.figure(figsize=(6,6))
plt.imshow(grid.permute(1,2,0))
plt.title("DCGAN Fake Samples")
plt.axis('off')
plt.show()

 ## 5. DCGAN Training Loss Curves



 Load your logged losses (saved as NumPy arrays during training).

In [ ]:
# Replace with your actual log paths
g_losses = np.load('logs/dcgan_g_losses.npy')
d_losses = np.load('logs/dcgan_d_losses.npy')
iterations = np.arange(len(g_losses)) * 100  # e.g. logged every 100 iters

plt.figure(figsize=(8,4))
plt.plot(iterations, g_losses, label='Generator Loss')
plt.plot(iterations, d_losses, label='Discriminator Loss')
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("DCGAN Training Curves")
plt.legend()
plt.grid(True)
plt.show()

 ## 6. CycleGAN Sample Comparisons



 Display saved sample images from 𝑋→𝑌 and 𝑌→𝑋 at iteration 400.

In [ ]:
def show_image(path, title):
    img = Image.open(path).convert('RGB')
    plt.figure(figsize=(4,4))
    plt.imshow(img)
    plt.title(title)
    plt.axis('off')
    plt.show()

show_image('output/cyclegan/sample-000400-X-Y.png', 'CycleGAN X→Y @400')
show_image('output/cyclegan/sample-000400-Y-X.png', 'CycleGAN Y→X @400')

 ## 7. Cycle Consistency Loss



 The cycle loss enforces \(G(F(y)) \approx y\) and \(F(G(x)) \approx x\), typically using L1:



 \[

   \mathcal{L}_{cycle}

   = \mathbb{E}_{x\sim X}\lVert F(G(x)) - x\rVert_1

   + \mathbb{E}_{y\sim Y}\lVert G(F(y)) - y\rVert_1.

 \]

 ## 8. Embedding TensorBoard in‑Notebook



 Launch TensorBoard directly in this notebook :contentReference[oaicite:5]{index=5}.

In [ ]:
# In a Jupyter cell, uncomment to launch:
# %load_ext tensorboard
# %tensorboard --logdir=output/vanilla  # or your CycleGAN logs